In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
from time import time

import numpy as np
import optuna
import pandas as pd
import seaborn as sns


from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.metrics import balanced_accuracy_score as bas

from objectives import RFobjective, RandomForestClassifier
from utils import Dataset

os.environ["KMP_WARNINGS"] = "FALSE"

In [ ]:
dataset = Dataset("../data/final_filtered_catalog_for_model_training_and_eval.pkl")

# window around the trigger for training models
w_min = 350
w_max = 500

In [ ]:
n_trials = 25
results =[]
i=1
wavetype = "alignedwaves"

optuna.logging.set_verbosity(optuna.logging.WARNING)

for till_test_exp in dataset.till_exp_nums:
    for rock_test_exp in dataset.rock_exp_nums:

        train = dataset.catalog[~dataset.catalog.expname.isin([till_test_exp, rock_test_exp])]
        test = dataset.catalog[dataset.catalog.expname.isin([till_test_exp, rock_test_exp])]

        x_train = np.stack(train[wavetype].values)
        x_test = np.stack(test[wavetype].values)
        y_train = train.labels.values
        y_test = test.labels.values

        testexps = (till_test_exp, rock_test_exp)
        rockevs = y_test.sum()
        tillevs = len(y_test) - y_test.sum()

        # limit x to first arrivals
        x_train = np.stack(x_train)[:,350:500]
        x_test = np.stack(x_test)[:,350:500]

        # Training and eval
        rfObjective = RFobjective(x_train, x_test, y_train, y_test)
        for objective, model in [
            (rfObjective, RandomForestClassifier),
        ]:
            t0 = time()
            print(i, model.__name__)
            study = optuna.create_study(direction="maximize", pruner=optuna.pruners.HyperbandPruner())
            study.optimize(objective, n_trials=n_trials)
            study.best_params
            clf = model(**study.best_params, n_jobs=-1)
            clf.fit(x_train, y_train)
            y_pred = clf.predict(x_test)

            # overall accuracy
            full_acc = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred)
            f1_macro = f1_score(y_test, y_pred, average="macro")
            f1_micro = f1_score(y_test, y_pred, average="micro")
            f1_weighted = f1_score(y_test, y_pred, average="weighted")
            bal_acc = bas(y_test, y_pred)
            roc_weighted = roc_auc_score(y_test, y_pred, average="weighted")
            roc_macro = roc_auc_score(y_test, y_pred, average="macro")
            roc_micro = roc_auc_score(y_test, y_pred, average="micro")

            # accuracy within each test experiment
            till_acc = accuracy_score(y_test[y_test==0], y_pred[y_test==0])
            rock_acc = accuracy_score(y_test[y_test==1], y_pred[y_test==1])

            results.append(
                {
                    "run_number": i,
                    "model": model.__name__,
                    "accuracy": full_acc,
                    "f1": f1,
                    "f1_macro": f1_macro,
                    "f1_weighted": f1_weighted,
                    "f1_micro": f1_micro,
                    "roc_macro": roc_macro,
                    "roc_weighted": roc_weighted,
                    "roc_micro": roc_micro,
                    "balanced_accuracy": bal_acc,
                    "till_accuracy": till_acc,
                    "rock_accuracy": rock_acc,
                    "test_exps": testexps,
                    "n_rock_evs": rockevs,
                    "n_till_evs": tillevs,
                    "classifier": clf,
                    "clf_params": study.best_params
                }
            )
            print(f"time: {(time()-t0):.2f}, till acc: {till_acc:.2f}, rock acc: {rock_acc:.2f}, ntill: {tillevs}, nrock: {rockevs}" )
        i += 1



In [ ]:
res = pd.DataFrame(results)

In [ ]:
res.till_accuracy.mean(), res.rock_accuracy.mean()

In [ ]:
res.groupby("model")["accuracy"].mean()

In [ ]:
res.describe()

In [ ]:
res["till_test_exp"] = res.test_exps.apply(lambda x: x[0])
res["rock_test_exp"] = res.test_exps.apply(lambda x: x[1])
res.to_pickle("../data/train_test_results.pkl")
res

In [ ]:
plt.scatter(res["n_till_evs"], res["till_accuracy"])
plt.scatter(res["n_rock_evs"], res["rock_accuracy"])

In [ ]:
res.groupby(["till_test_exp","model"])["till_accuracy"].mean()

In [ ]:
res.groupby(["rock_test_exp","model"])["rock_accuracy"].mean()

In [ ]:
plt.figure(figsize=(30,5))
sns.boxplot(data=res, x="run_number", y="balanced_accuracy")

In [ ]:
acc_by_testexps = pd.DataFrame(res.groupby("test_exps")["balanced_accuracy"].mean()).reset_index()

In [ ]:
acc_by_test_exp = res.groupby("run_number").aggregate(
    {
        "balanced_accuracy": np.mean,
        "till_test_exp": lambda x: list(x)[0],
        "rock_test_exp": lambda x: list(x)[0]
    }
)

In [ ]:
acc_by_test_exp = acc_by_test_exp.pivot(index="till_test_exp", columns="rock_test_exp", values="balanced_accuracy")

In [ ]:
plt.figure(figsize=(9,7))
sns.heatmap(
    acc_by_test_exp,
    cmap="seismic",
    vmin=0, vmax=1,
    annot=True, fmt=".2f"
)
plt.title("Mean accuracy by test set experiment pair (till, rock)")
plt.ylabel("Till experiment #")
plt.xlabel("Rock experiment #")
plt.show()

In [ ]:
res.groupby("rock_test_exp")["rock_accuracy"].mean().plot.bar()

In [ ]:
res.groupby("rock_test_exp")["f1"].mean().plot.bar()

In [ ]:
res.groupby("till_test_exp")["till_accuracy"].mean().plot.bar()